In [0]:
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
ls data

men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y=df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)


In [0]:
df ['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
test = {'key': 'value'}
test['key']

str(test)

"{'key': 'value'}"

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)[0]['value'][0]

'Men'

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan' : return output_dict

  features =  literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict


df['features_parsed'] = df['features'].map(parse_features)

In [0]:
keys = set()

df['features_parsed'].map( lambda x: keys.update( x.keys()))

len(keys)

476

In [0]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [0]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: key in feats[key] if key in feats else np.nan)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_item spec attr - style (ebay)_cat', 'feat_width_cat',
       'feat_color family_cat', 'feat_inseam_cat', 'feat_charger included_cat',
       'feat_product in inches (l x w x h)_cat',
       'feat_origin of components_cat', 'feat_toe type_cat',
       'feat_licensed_cat', 'feat_removable hood_cat'],
      dtype='object', length=1002)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[ False ==  df[get_name_feat(key)].isnull() ].shape[0]/df.shape[0]*100

In [0]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df.shape[0]

18280

In [0]:
df ['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df ['feat_color_cat'] = df['feat_color'].factorize()[0]
df ['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df ['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df ['feat_material_cat'] = df['feat_material'].factorize()[0]

df ['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df ['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys: #44'
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [0]:
df['brand'] = df['brand'].map(lambda x : str(x).lower())
df[ df.brand == df.feat_brand ].shape

(0, 1002)

In [0]:
feats = ['']

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(['brand_cat'], model)

(-57.31964683129153, 4.171237265778642)

In [0]:
feats_cat = [ x for x in df.columns if 'cat' in x ]
feats_cat

['categories',
 'brand_cat',
 'feat_fabrication',
 'feat_multi pack indicator',
 'feat_shoe category',
 'feat_catalog',
 'feat_location - city/state',
 'feat_certifications and listings',
 'feat_recommended location',
 'feat_location - country',
 'feat_clothing category',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_front_cat',
 'feat_sleeve style_cat',
 'feat_weather resistant_cat',
 'feat_is weather-resistant_cat',
 'feat_main color_cat',
 'feat_nike id_cat',
 'feat_machine washable_cat',
 'feat_heat zones_cat',
 'feat_military style_cat',
 'feat_lens tint_cat',
 'feat_eye protection type_cat',
 'feat_frame depth_cat',
 'feat_lens socket width_cat',
 'feat_manufacturer sku_cat',
 'feat_fabric care instructions_cat',
 'feat_fabrication_cat',
 'feat_series name_cat',
 'feat_work shoes_cat',
 'feat_stability_cat',
 'feat_sizearm_cat',
 'feat_country/ organization_cat',
 

In [0]:
feats = ['brand_cat', 'feat_color_cat',	'feat_adjustable_cat', 'feat_size_cat', 'feat_shape_cat', 'feat_manufacturer part number_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [0]:
X = df [feats].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)
print(result)

perm = PermutationImportance(m, random_state = 1).fit(X, y);
eli5.show_weights(perm, feature_names = feats)

(-58.208016411760056, 4.79327170775813)


Weight,Feature
0.2415 ± 0.0079,brand_cat
0.0776 ± 0.0076,feat_color_cat
0.0304 ± 0.0061,feat_adjustable_cat
0.0149 ± 0.0017,feat_shape_cat
0.0094 ± 0.0034,feat_material_cat
0.0084 ± 0.0045,feat_style_cat
0.0042 ± 0.0004,feat_size_cat
0.0017 ± 0.0003,feat_manufacturer part number_cat
0.0000 ± 0.0000,feat_sport_cat


In [0]:
df['brand'].value_counts(normalize = True)

nike                         0.097210
puma                         0.033315
ralph lauren                 0.028775
vans                         0.021116
new balance                  0.020295
                               ...   
fit & fresh                  0.000055
john ashford                 0.000055
earrings-midwestjewellery    0.000055
augusta sportswear           0.000055
marc ny                      0.000055
Name: brand, Length: 1732, dtype: float64

In [0]:
df[df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'sport': 'soccer', 'condition': 'new with tags', 'type': 'socks'},
       {},
       {'sport': 'baseball & softball, basketball, camping & hiking, cycling, fishing, fitness, football, gymnastics, hockey, running', 'main color': 'white/blue', 'style': 'shirts & tops', 'sleeve length': 'short sleeve', 'condition': 'new with tags'},
       {'sport': 'baseball & softball', 'type': 'cleats', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'color': 'total orange/white/game royal/dark obsidian', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '620465801', 'brand': 'nike', 'shoe width': 'medium (d, m)'}],
      dtype=object)

In [0]:
df['feat_age group'].value_counts()

False    5054
Name: feat_age group, dtype: int64

In [163]:
!git add matrix_one/day5.ipynb
!git status  #-> patrzymy czy aby na pewno wysyłamy dobry plik
!git commit -m "Zaawansowany model uczenia maszynowego day5"

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	new file:   matrix_one/day5.ipynb

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   .gitignore
	modified:   matrix_one/day3.ipynb
	modified:   matrix_one/day4.ipynb


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@18c392b24264.(none)')


In [0]:
!git config --global user.email "eeyore774@gmail.com"
  git config --global user.name "Eeyore

In [162]:
!git push -u origin master

Branch 'master' set up to track remote branch 'master' from 'origin'.
Everything up-to-date
